In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import os

In [31]:
#Reverse the data
data = pd.read_csv('data.csv').iloc[::-1].reset_index(drop=True)

In [35]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data.drop(columns=['timestamp']).values, dtype=torch.float32)
    def __len__(self):
        return len(self.data)-25
    def __getitem__(self, idx):
        x = self.data[idx:idx+25, :]
        y = self.data[idx+25, list(range(0, 80, 5))]
        return x, y
dataset = MyDataset(data)        

In [42]:
x,_=dataset[1]
x.size()

torch.Size([25, 130])

In [45]:
#Split the data
train_set, val_set = torch.utils.data.random_split(dataset, [0.8,0.2])
val_load = DataLoader(val_set, batch_size=len(val_set), shuffle=False)
train_load = DataLoader(train_set, batch_size=64, shuffle=True)
x_val, y_val = next(iter(val_load))

In [43]:
#Batch first positional encoding
class position_encoding(nn.Module):
    def __init__(self, d_model, max_len=25):
        super(position_encoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [44]:
class attention_model(nn.Module):
    def __init__(self):
        super(attention_model, self).__init__()
        self.bn1 = nn.BatchNorm2d(130)
        self.position_encoder = position_encoding(130)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=130, nhead=5,batch_first=True)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=2)
        self.fc = nn.Linear(130, 17)
    def forward(self, x):
        x = self.bn1(x)
        x = self.position_encoder(x)
        x = self.decoder(x)
        x = self.fc(x)
        return x
model1 = attention_model()

In [46]:
def loss(x, y_pred, y_true):
    split = F.softmax(y_pred, dim=1)
    x_value = x[:, 24, list(range(0, 80, 5))]
    ratio = y_true/x_value
    gain = ratio * split[:,:-1].sum(dim=1).meam()
    return -gain

In [47]:
@torch.no_grad()
def val_loss():
    model1.eval()
    preds = model1(x_val)
    model1.train()
    return loss(x_val, preds, y_val).item()

In [50]:
optimizer = optim.Adam(model1.parameters(), lr=0.01, weight_decay=0.001)
scheduler = optim.scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
def train_one_epoch(epoch):
    model1.train()
    total_loss = 0
    for i in train_load:
        model1.train()
        optimizer.zero_grad()
        features, targets = i
        features=features.to('cuda')
        targets=targets.to('cuda')
        preds = model1(features)
        loss = loss(features, preds, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()*features.shape[0]
    scheduler.step(val_loss())
    val=float(val_loss())
    writer.add_scalar('validation loss', val, epoch)
    return f"loss: {total_loss / len(train_set)}\n val_loss: {val}\n"